# Using the grid search function with fsm. 

Thie notebook demonstrates how to use the grid search function with the fsm algorithm. 

Import the functions contained in funcs.py

In [1]:
%load_ext autoreload
%autoreload 2 
import sys
sys.path.append("../..") 
import scripts.funcs as fun

/Users/jkingslake/Documents/science/meltwater_routing/BFRN_meltwater/python/notebooks/new_features/../../scripts/funcs.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


To run `gridSearch` you provide the function as the first parameter and as many other paremeters as you want to vary over after that. 

For example, to run our fsm function (`fsm_xarray`) while varying the x position of the center of the square of melt and the magnitude of the melt, do the following:

In [5]:
results = fun.gridSearch(fun.fsm_xarray, x_center_of_melt = [812500.0, 817500.0], melt_magnitude=[0.2, 2])

  0%|          | 0/4 [00:00<?, ?it/s]

This produces an xarray.dataset with the results of the multi-dimensional gridsearch arranged in a useful way:

In [6]:
results

<xarray.Dataset>
Dimensions:                 (x_center_of_melt: 2, melt_magnitude: 2, x: 1024,
                             y: 3072, bounds_index: 4)
Coordinates:
  * x_center_of_melt        (x_center_of_melt) float64 8.125e+05 8.175e+05
  * melt_magnitude          (melt_magnitude) float64 0.2 2.0
  * x                       (x) float64 8e+05 8e+05 ... 8.327e+05 8.327e+05
  * y                       (y) float64 2e+06 2e+06 ... 1.902e+06 1.902e+06
    spatial_ref             int64 0
Dimensions without coordinates: bounds_index
Data variables:
    water_depth             (y, x, x_center_of_melt, melt_magnitude) float64 <COO: nnz=43554, fill_value=0.0>
    dem                     (y, x, x_center_of_melt, melt_magnitude) float32 dask.array<chunksize=(3072, 1024, 1, 2), meta=np.ndarray>
    melt                    (y, x, x_center_of_melt, melt_magnitude) float32 <COO: nnz=97344, fill_value=0.0>
    bounds                  (bounds_index, x_center_of_melt, melt_magnitude) float64 ...
    x_center_of_mass        (x_center_of_melt, melt_magnitude) float64 8.163e...
    y_center_of_mass        (x_center_of_melt, melt_magnitude) float64 1.938e...
    L                       (x_center_of_melt, melt_magnitude) float64 6.744e...
    total_GL_flux_response  (x_center_of_melt, melt_magnitude) float64 dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    AREA_OR_POINT:       Area
    PROCESSING_HISTORY:  2024-03-14 01:07:40 UTC | RichDEM v2.2.9 (hash=41503...
    TIFFTAG_DATETIME:    2024-03-14 01:07:40 UTC
    TIFFTAG_SOFTWARE:    RichDEM v2.2.9 (hash=415032db2f303721, compiled=2024...
    _FillValue:          -9999.0
    scale_factor:        1.0
    add_offset:          0.0

The main result is in the `water_depth` variable. This is a series of maps (one for each parameter combination) of where the water accumulated for that particular set of parameters. For example:

In [13]:
import hvplot.xarray

results.water_depth.as_numpy().hvplot(x ='y', y = 'x', cmap='Blues', clim=(0,1), width = 500, height = 500, aspect='equal')

BokehModel(combine_events=True, render_bundle={'docs_json': {'b344414c-14f8-43c2-b4ec-930efcbf2c27': {'version…

Now we can save the results as a netcdf. By default some of the arrays are output by gridSearch are sparse arrays, so to save to netcdf you ned ot first convert them to dense arrays. 


In [11]:
results = results.as_numpy()
results.to_netcdf("results_small_test.nc")

In [18]:
fun.map_water_depth(results)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e4e68022-c1c5-43d0-ae65-54de2d94ddc4': {'version…

THe xample below loops over the x and y positions of the melt square, and uses a larger DEM. The result is quite large.

In [1]:
results = fun.gridSearch(fun.fsm_xarray, dem_filename = ["/Users/jkingslake/Documents/science/meltwater_routing/big_data/boudouin_west_1_coarsen6_largerROI.tif"],
                         x_center_of_melt=np.arange(9.000e5,9.4e5,5000), 
                         y_center_of_melt=np.arange(1.866e6, 1.95e6, 5000), 
                         melt_magnitude=[0.2])

NameError: name 'fun' is not defined